In [31]:
import galois # Need to install galois package: pip install galois
import numpy as np
import sys
sys.path.append('../../doubling-CSST/')
from convert_alist import readAlist
alistDirPath = "../../doubling-CSST/alistMats/GO03_self_dual/"
# The following are the parameters of the self-dual code included in the alistDirPath
length_dist_dict = {4:2, 6:2, 8:2, 10:2, 12:4, 14:4, 16:4, 18:4, 20:4, 22:6, 24:6, 26:6, 28:6, 30:6, 32:8, 34:6, 36:8, 38:8, 40:8, 42:8, 44:8, 46:8, 48:8, 50:8, 52:10, 54:8, 56:10, 58:10, 60:12, 62:10, 64:10}

n = 12 # all even lengths from 4 to 64
d = length_dist_dict[n]
F2 = galois.GF(2)
alistFilePath = alistDirPath + "n" + str(n) + "_d" + str(d) + ".alist"

GenMat = F2(readAlist(alistFilePath))
print("Generator matrix of the [n=%s,k=%s,d=%s] self-dual code:" % (n, n//2, d))
print(GenMat)

Generator matrix of the [n=12,k=6,d=4] self-dual code:
[[0 1 1 1 0 0 1 1 1 0 0 0]
 [0 1 1 0 0 1 1 1 0 0 1 0]
 [1 0 0 0 0 1 1 1 0 0 1 1]
 [0 0 1 0 0 1 1 0 0 1 1 1]
 [0 0 1 1 1 0 0 0 0 1 1 1]
 [0 1 1 1 0 0 1 0 0 1 1 0]]


In [34]:
import numpy as np
from ldpc.bplsd_decoder import BpLsdDecoder
import scipy.sparse

H_standard_np = np.array(GenMat, dtype=int)
H_sparse = scipy.sparse.csr_matrix(H_standard_np)
print(H_sparse)

bp_osd = BpLsdDecoder(
    H_sparse,
    error_rate = 0.1,
    bp_method = 'product_sum',
    max_iter = 2,
    schedule = 'serial',
    lsd_method = 'lsd_cs',
    lsd_order = 0
)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (1, 1)	1
  (1, 2)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 10)	1
  (2, 0)	1
  (2, 5)	1
  (2, 6)	1
  (2, 7)	1
  (2, 10)	1
  (2, 11)	1
  (3, 2)	1
  (3, 5)	1
  (3, 6)	1
  (3, 9)	1
  (3, 10)	1
  (3, 11)	1
  (4, 2)	1
  (4, 3)	1
  (4, 4)	1
  (4, 9)	1
  (4, 10)	1
  (4, 11)	1
  (5, 1)	1
  (5, 2)	1
  (5, 3)	1
  (5, 6)	1
  (5, 9)	1
  (5, 10)	1


In [27]:
syndrome = np.random.randint(size=H_sparse.shape[0], low=0, high=2).astype(np.uint8)

print(f"Syndrome: {syndrome}")
decoding = bp_osd.decode(syndrome)
print(f"Decoding: {decoding}")
decoding_syndrome = H_sparse@decoding % 2
print(f"Decoding syndrome: {decoding_syndrome}")

Syndrome: [1 1 0 1 0 0]
Decoding: [0 1 0 0 0 0 1 1 0 0 0 0]
Decoding syndrome: [1 1 0 1 0 0]
